# Frame-by-frame video inference and annotated output

**Objective:** Frame-by-frame video inference and annotated output

This notebook is designed for practical YOLO workflows. It includes safe defaults and clear placeholders.


In [ ]:
# ==============================
# ROKAF AI Competition Prep Notebook
# ==============================
# Purpose:
# - Provide a clean, competition-ready template
# - Demonstrate key techniques with clear, step-by-step explanations
#
# Notes:
# - If you run this notebook in a fresh environment, install dependencies below.
# - The code is intentionally verbose and heavily commented for learning and review.
#
# Author: Jungmyung Lee
# ==============================

# Optional: install dependencies (uncomment if needed)
# !pip -q install numpy matplotlib ultralytics opencv-python

import os
import time
import numpy as np

try:
    from ultralytics import YOLO
except ImportError as e:
    raise ImportError(
        "Ultralytics is not installed. Uncomment the pip install line above and re-run."
    ) from e

print("✅ Ultralytics YOLO imported successfully.")


## Video inference pipeline

This is a template for frame-by-frame processing. Replace the input path with your real video.


In [ ]:
import cv2

model = YOLO("yolov8n.pt")

# Create a tiny demo video to keep this notebook runnable anywhere
os.makedirs("data_demo", exist_ok=True)
demo_vid = "data_demo/demo_video.mp4"

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
vw = cv2.VideoWriter(demo_vid, fourcc, 10, (640, 480))
for i in range(30):
    frame = np.zeros((480,640,3), dtype=np.uint8)
    cv2.putText(frame, f"t={i}", (50,100), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (255,255,255), 3)
    vw.write(frame)
vw.release()

cap = cv2.VideoCapture(demo_vid)
out_path = "data_demo/demo_video_annotated.mp4"
vw = cv2.VideoWriter(out_path, fourcc, 10, (640,480))

frame_count = 0
t0 = time.time()

while True:
    ok, frame = cap.read()
    if not ok:
        break

    results = model.predict(source=frame, conf=0.25, iou=0.7, verbose=False)
    annotated = results[0].plot()
    vw.write(annotated)
    frame_count += 1

cap.release()
vw.release()

dt = time.time() - t0
print(f"✅ Processed {frame_count} frames in {dt:.2f}s ({frame_count/max(dt,1e-9):.1f} FPS).")
print("Saved:", out_path)
